# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [4]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [28]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to check to see what the list of event data rows will look like and the total number of rows 
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [22]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [23]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as ex:
    print(ex)

#### Set Keyspace

In [24]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as ex:
    print(ex)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [25]:
# Create a song_info table for the existing namespace
query = "CREATE TABLE IF NOT EXISTS music_app_session_history "
query = query + "(session_id int, item_in_session int, artist_name text, song_length float, song_title text, PRIMARY KEY (session_id, item_in_session))"                   

try:
    session.execute(query)
except Exception as e:
    print(e)

In [26]:
file = 'event_datafile_new.csv'

# Fill the newly created music_app_session_history table with the value inside the file: event_datafile_new.csv
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_session_history (session_id, item_in_session, artist_name, song_length, song_title) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [27]:
# Check if the data was correctly imported by querying it by the session_id and item_in_session
query = "SELECT artist_name, song_length, song_title from music_app_session_history WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
dataframe = pd.DataFrame(rows._current_rows)
dataframe

Row(artist_name='Faithless', song_length=495.30731201171875, song_title='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
# Create another table, but now to query per user and session (not just session).
createTableQuery = "CREATE TABLE IF NOT EXISTS music_app_user_history "
createTableQuery = createTableQuery + "(artist_name text, song_name text, item_in_session int, first_name text, last_name text, user_id int, session_id int, PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    session.execute(createTableQuery)
except Exception as e:
    print(e)

# Populate the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        insertQuery = "INSERT INTO music_app_user_history (artist_name, song_name, item_in_session, first_name, last_name, user_id, session_id) "
        insertQuery = insertQuery + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(insertQuery, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))
        except Exception as e:
            print(e)

# Query using the user_id and session_id PK (the clustered column is item_in_session)
query = "SELECT artist_name, song_name, first_name, last_name FROM music_app_user_history WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

dataframe = pd.DataFrame(rows._current_rows)
dataframe

In [ ]:
# Create another table, but now it needs to query by a specific song
createTableQuery = "CREATE TABLE IF NOT EXISTS song_app_history "
createTableQuery = createTableQuery + "(song_name text, first_name text, last_name text, user_id int, PRIMARY KEY(song_name, user_id))"

try:
    session.execute(createTableQuery)
except Exception as e:
    print(e)

# Populate the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        # Query using 
        insertQuery = "INSERT INTO song_app_history (song_name, first_name, last_name, user_id) "
        insertQuery = insertQuery + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(insertQuery, (line[9], line[1], line[4], int(line[10])))
        except Exception as e:
            print(e)

# Query by a song name
query = "SELECT song_name, first_name, last_name FROM song_app_history WHERE song_name='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

dataframe = pd.DataFrame(rows._current_rows)
dataframe

### Drop the tables before closing out the sessions

In [4]:
# Drop all tables before shutdown
try:
    session.execute("DROP TABLE IF EXISTS music_app_session_history")
    session.execute("DROP TABLE IF EXISTS music_app_user_history")
    session.execute("DROP TABLE IF EXISTS song_app_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()